## Лабораторна робота №4

Обробка природної мови

In [2]:
import gensim

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('c:/boxroom/data-science/L07/nlp_ukr/model.bin', binary=True)

In [4]:
len(model)

942071

In [5]:
# В скількох розмірностях представлено кожне слово
model.vector_size

100

In [6]:
# Пошук top N найбільш схожих слів до ключового слова
model.most_similar(positive=['університет'], topn = 3)

[('коледж', 0.872685968875885),
 ('політехнічний', 0.8209682703018188),
 ('інститут', 0.8206391334533691)]

In [7]:
# Пошук зайвого слова у списку
model.doesnt_match('сніданок космос обід вечеря'.split())

'космос'

In [8]:
# Прояв синтаксично-семантичних закономірностей
model.most_similar(positive=['жінка','король'], negative=['чоловік'], topn = 1)

[('королева', 0.7483499050140381)]

In [9]:
model.most_similar(positive=['київ','німеччина'], negative=['берлін'], topn = 1)

[('україна', 0.7106590270996094)]

---

# Сумаризація тексту

Вибрати свій варіант

https://naurok.com.ua/teksti-dlya-perekaziv-z-ukra-nsko-movi-profesiyno-zorientovani-kuhar-konditer-200658.html

In [13]:
import spacy

In [14]:
!python -m spacy download uk_core_news_sm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/14.9 MB 2.3 MB/s eta 0:00:07
     ---------------------------------------- 0.2/14.9 MB 2.1 MB/s eta 0:00:08
      --------------------------------------- 0.3/14.9 MB 2.7 MB/s eta 0:00:06
     - -------------------------------------- 0.5/14.9 MB 3.0 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/14.9 MB 3.9 MB/s eta 0:00:04
     --- ------------------------------------ 1.2/14.9 MB 5.0 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/14.9 MB 5.5 MB/s eta 0:00:03
     ---- ----------------------------------- 1.8/14.9 MB 5.8 MB/s eta 0:00:03
     ----- ---------------------------------- 2.2/14.9 MB 6.2 MB/s eta 0:00:03
     ------ --------------------------------- 2.6/14.9 MB 6.4 MB/s eta 0:00:02
     -------- ------------------------------- 3.1/14.9 MB 7.0 MB/

C:\Users\tedim\miniconda3\Lib\site-packages\torch\cuda\__init__.py:740: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [15]:
import uk_core_news_sm
nlp = uk_core_news_sm.load()

In [16]:
from nltk.corpus import stopwords
from nltk import sent_tokenize
from heapq import nlargest

In [17]:
text = """
Хліб в українській обрядовості
Здавна народ України славився своєю гостинністю, бо «веселий гість – дому радість». Споконвіку на столі, застеленому барвистим вишитим рушником, лежав хліб, а поруч нього – сіль.
Хліб – один з найдавніших і найважливіших символів української обрядовості. При цьому його ужиткова функція відходить на другий план, а головною стає символічність. Усі види весільного печива виражали ідею єдності родини, міцності шлюбу, продовження роду, благополуччя і злагоди в сім`ї. Чільне значення мав хліб у родильній та поховальній обрядовості, у проведенні календарних і трудових свят. Традиції, пов`язані з хлібом, який уособлює повагу до праці, кращі моральні засади, продовжують жити і сьогодні.
В Україні існує багато видів обрядового хлібу. Сюди слід віднести верч – весільний обрядовий хліб. Ним обмінювалися на весіллі родини: молодий привозив верч до тещі, а молода – для свекрухи. Дивень – обрядовий хліб, аналогічний за своїм ритуальним призначенням калачеві. Його виробляли з плетінки втроє чи вчетверо, а потім молода дивилась на молодого, звідки, очевидно, й походить його назва. Жайворонки – печиво, що випікали на свято Сорока святих. Їх пекли сорок штук, їли протягом дня, причому обов`язково діти. Вони бігали по селу, підіймали вгору «пташок» та промовляли нехитрі заклики. За повір`ям, ці дії прискорювали повернення птахів із вирію, тобто наближали початок весни. Це обрядове печиво символізувало весну, зустріч із рідною землею, гостинність, відродження природи.
Коровай – обрядовий весільний хліб, поширений серед багатьох слов`янських народів. Коровай символізував гідність, достаток, красу молодих на весіллі. Ще й досі при виготовленні короваю дотримуються звичаїв, що йдуть із глибокої давнини. Коровай як символ завершеності аграрного року, випікали й на обжинки: разом із обжинковим снопом він є одним з найважливіших символів – атрибутів свята врожаю й майбутнього достатку.
До найважливіших весняних обрядових хлібів належить пасха, яку готували до Великодня. Пасху обов`язково святили в церкві разом з писанками, крашанками та іншими стравами святкового столу. Пасху, писанки й крашанки наділяли магічними властивостями та приписували їм цілющу силу. Пасха мала символічне значення Бога, єднання людей із вищим розумом, слугувала обереговим знаком перед силами зла. Весільний коровай, пасхальне печиво, святочні пироги вважаються символами астральної тріади: місяця, сонця і зорі або місяця, сонця і Господа. Про це свідчать весільні пісні, щедрівки, колядки, веснянки. До речі, ідучи на весілля, люди несли господарю два хліби (символи сонця і місяця), а господині несли по мисці білого борошна (просліджується подвійна символіка – ясність білого борошна, округлість миски).
Особливою стравою українського народу були пиріжки. Готували пиріжки з різноманітною начинкою: м`ясом, рибою, грибами, сиром, горохом, капустою, маком, яблуками. Особливо популярними були пиріжки з калиною. Це символічна страва, що пов`язана з культом місяця і формою нагадує півмісяць. Свою символіку мали пироги з маком - подвійний символ місяця й зірок, що знаменував ріст і плодючість.
Хліб в Україні в давнину мав надзвичайну шану й вважався святим. Він був символом святості, радості, щастя, здоров`я та багатства. Люди вірили, що з хлібом у домі присутні добрі душі.
"""
text = ' '.join(text.split('\n'))
text

' Хліб в українській обрядовості Здавна народ України славився своєю гостинністю, бо «веселий гість – дому радість». Споконвіку на столі, застеленому барвистим вишитим рушником, лежав хліб, а поруч нього – сіль. Хліб – один з найдавніших і найважливіших символів української обрядовості. При цьому його ужиткова функція відходить на другий план, а головною стає символічність. Усі види весільного печива виражали ідею єдності родини, міцності шлюбу, продовження роду, благополуччя і злагоди в сім`ї. Чільне значення мав хліб у родильній та поховальній обрядовості, у проведенні календарних і трудових свят. Традиції, пов`язані з хлібом, який уособлює повагу до праці, кращі моральні засади, продовжують жити і сьогодні. В Україні існує багато видів обрядового хлібу. Сюди слід віднести верч – весільний обрядовий хліб. Ним обмінювалися на весіллі родини: молодий привозив верч до тещі, а молода – для свекрухи. Дивень – обрядовий хліб, аналогічний за своїм ритуальним призначенням калачеві. Його виро

In [18]:
doc = nlp(text)

In [19]:
sentences = list(doc.sents)
for i,s in enumerate(sentences):
    print('{:3d}'.format(i+1), s)

  1  Хліб в українській обрядовості Здавна народ України славився своєю гостинністю, бо «веселий гість – дому радість».
  2 Споконвіку на столі, застеленому барвистим вишитим рушником, лежав хліб, а поруч нього – сіль.
  3 Хліб – один з найдавніших і найважливіших символів української обрядовості.
  4 При цьому його ужиткова функція відходить на другий план, а головною стає символічність.
  5 Усі види весільного печива виражали ідею єдності родини, міцності шлюбу, продовження роду, благополуччя і злагоди в сім`ї.
  6 Чільне значення мав хліб у родильній та поховальній обрядовості, у проведенні календарних і трудових свят.
  7 Традиції, пов`язані з хлібом, який уособлює повагу до праці, кращі моральні засади, продовжують жити і сьогодні.
  8 В Україні існує багато видів обрядового хлібу.
  9 Сюди слід віднести верч – весільний обрядовий хліб.
 10 Ним обмінювалися на весіллі родини: молодий привозив верч до тещі, а молода – для свекрухи.
 11 Дивень – обрядовий хліб, аналогічний за своїм 

In [20]:
sentence_weights = {s: len(s) for s in sentences}
sorted_sentences = sorted(sentence_weights, key=lambda x: sentence_weights[x], reverse=True)
for i,s in enumerate(sorted_sentences[:3]):
    print('{:3d}'.format(i+1), '[{:03d}]'.format(sentence_weights[s]), s)
# [(sentence_weights[s], s) for s in sorted_sentences][:3]

  1 [039] До речі, ідучи на весілля, люди несли господарю два хліби (символи сонця і місяця), а господині несли по мисці білого борошна (просліджується подвійна символіка – ясність білого борошна, округлість миски).
  2 [030] Коровай як символ завершеності аграрного року, випікали й на обжинки: разом із обжинковим снопом він є одним з найважливіших символів – атрибутів свята врожаю й майбутнього достатку.
  3 [025] Весільний коровай, пасхальне печиво, святочні пироги вважаються символами астральної тріади: місяця, сонця і зорі або місяця, сонця і Господа.


In [21]:
summary = ' '.join([sent.text for sent in nlargest(3, sentence_weights, key=sentence_weights.get)])
print('Переказ:')
print(summary)

Переказ:
До речі, ідучи на весілля, люди несли господарю два хліби (символи сонця і місяця), а господині несли по мисці білого борошна (просліджується подвійна символіка – ясність білого борошна, округлість миски). Коровай як символ завершеності аграрного року, випікали й на обжинки: разом із обжинковим снопом він є одним з найважливіших символів – атрибутів свята врожаю й майбутнього достатку. Весільний коровай, пасхальне печиво, святочні пироги вважаються символами астральної тріади: місяця, сонця і зорі або місяця, сонця і Господа.
